In [2]:
import numpy as np
import tensorflow as tf

from keras import backend as K
from keras.layers import Input, Dense, Lambda, Flatten, Activation, Merge, Concatenate, Add
from keras import layers
from keras.layers.merge import concatenate
from keras.models import Model, Sequential
from keras.objectives import binary_crossentropy
from keras.callbacks import LearningRateScheduler
from keras.models import load_model

from models import vgg
from models.cvae import CVAE
from utils.angles import deg2bit, bit2deg
from utils.losses import mad_loss_tf, cosine_loss_tf, von_mises_loss_tf, maad_from_deg
from utils.losses import gaussian_kl_divergence_tf, gaussian_kl_divergence_np
from utils.losses  import von_mises_log_likelihood_tf, von_mises_log_likelihood_np
from utils.towncentre import load_towncentre
from utils.experiements import get_experiment_id

#### TownCentre data

In [3]:
xtr, ytr_deg, xval, yval_deg, xte, yte_deg = load_towncentre('data/TownCentre.pkl.gz', 
                                                             canonical_split=True,
                                                             verbose=1)
image_height, image_width = xtr.shape[1], xtr.shape[2]
ytr_bit = deg2bit(ytr_deg)
yval_bit = deg2bit(yval_deg)
yte_bit = deg2bit(yte_deg)

image_height, image_width, n_channels = xtr.shape[1:]
flatten_x_shape = xtr[0].flatten().shape[0]
phi_shape = yte_bit.shape[1]

Number of train samples: 6916
Number of validation samples: 874
Number of test samples: 904


#### Notation

$x$ - image,

$\phi$ - head angle,

$u$ - hidden variable

#### Prior network

$ p(u|x) \sim \mathcal{N}(\mu_1(x, \theta), \sigma_1(x, \theta)) $

#### Encoder network

$ q(u|x,\phi) \sim \mathcal{N}(\mu_2(x, \theta), \sigma_2(x, \theta)) $

#### Sample  $u \sim \{p(u|x), q(u|x,\phi) \}$

#### Decoder network

$p(\phi|u,x) \sim \mathcal{VM}(\mu(x,u,\theta''), \kappa(x,u,\theta'')) $

In [4]:
n_u = 8

cvae_model = CVAE(n_hidden_units=n_u)

#### Training

In [5]:
import keras
from utils.custom_keras_callbacks import SideModelCheckpoint

cvae_best_ckpt_path = 'logs/cvae.full_model.best.weights.hdf5'


model_ckpt_callback = keras.callbacks.ModelCheckpoint(cvae_best_ckpt_path,
                                                      monitor='val_loss',
                                                      mode='min',
                                                      save_best_only=True,
                                                      save_weights_only=True,  
                                                      verbose=1)

In [6]:
cvae_model.full_model.fit([xtr, ytr_bit], [ytr_bit], batch_size=10, epochs=50, validation_data=([xval, yval_bit], yval_bit),
                   callbacks=[model_ckpt_callback])

Train on 6916 samples, validate on 874 samples
Epoch 1/1
6916/6916 [==============================] - 85s - loss: 1.8283 - val_loss: 2.2512


#### Predictions using decoder part

$ \phi_i = \mu(x_i,u_i,\theta'') $

In [10]:
best_model = CVAE(n_hidden_units=n_u)
best_model.full_model.load_weights(cvae_best_ckpt_path)

In [12]:
results = dict()
results['train'] = best_model.evaluate(xtr, ytr_deg, 'train')
results['validation'] = best_model.evaluate(xval, yval_deg, 'validation')
results['test'] = best_model.evaluate(xte, yte_deg, 'test')

MAAD error (train) : 80.638380 ± 0.699361SEM
ELBO (train) : -2.115995 ± 0.014231SEM
log-likelihood (train) : -2.120826±0.014496SEM
MAAD error (validation) : 87.788975 ± 1.932653SEM
ELBO (validation) : -2.307066 ± 0.040839SEM
log-likelihood (validation) : -2.303484±0.042332SEM
MAAD error (test) : 90.097177 ± 1.983286SEM
ELBO (test) : -2.264259 ± 0.038184SEM
log-likelihood (test) : -2.310206±0.040678SEM
